In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder,MinMaxScaler
from sklearn.linear_model import LogisticRegression
import numpy as np

In [2]:
def group_pivot(labelgroup, yvalue, dataset, SZOnly = False):
    if SZOnly:
        dataset = dataset.loc[dataset.Diagnosis == 0]
    
    grouped = (dataset.groupby([labelgroup])[yvalue].value_counts(normalize = True).rename('percentage').reset_index())
    pivot = pd.pivot_table(grouped, index = labelgroup, columns = yvalue, values = 'percentage', aggfun = 'sum')
    return pivot

In [27]:
data = pd.read_csv('/Users/dahaixing/Documents/Coursework/DeepLearning/MLCW/MachineLearningCW/MS4S16_Dataset.csv')

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Diagnosis           5000 non-null   int64  
 1   Anhedonia           5000 non-null   float64
 2   Apathy              5000 non-null   float64
 3   Appetite            5000 non-null   float64
 4   Concentration       5000 non-null   float64
 5   Content             5000 non-null   float64
 6   Delay               5000 non-null   object 
 7   Delusion            5000 non-null   float64
 8   Dep_Mood            5000 non-null   float64
 9   Focus               5000 non-null   float64
 10  Hallucination       5000 non-null   float64
 11  Housing             5000 non-null   object 
 12  Intrusive_Thoughts  4170 non-null   float64
 13  Participant         5000 non-null   int64  
 14  Passive             5000 non-null   float64
 15  Pregnant            2762 non-null   float64
 16  Psycho

In [29]:
def data_impu(data):
    data.fillna(data.mean(), inplace=True)
    mean_tired = data['Tired'].replace(['NaN','inf'], pd.np.nan).astype(float).mean()
    mean_tired = data['Tired'].replace([pd.np.nan, pd.np.inf], pd.np.nan).mean()
    data['Tired'].replace(['NaN','inf'], mean_tired)
    data.replace([pd.np.nan, pd.np.inf], [mean_tired, mean_tired], inplace=True)
    return data

def data_remove(data):
    data.fillna(data.mean(), inplace=True)
    data = data.replace([np.inf, -np.inf], np.nan)
    data = data.dropna()
    return data
    

In [30]:
#data = data_remove(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Diagnosis           5000 non-null   int64  
 1   Anhedonia           5000 non-null   float64
 2   Apathy              5000 non-null   float64
 3   Appetite            5000 non-null   float64
 4   Concentration       5000 non-null   float64
 5   Content             5000 non-null   float64
 6   Delay               5000 non-null   object 
 7   Delusion            5000 non-null   float64
 8   Dep_Mood            5000 non-null   float64
 9   Focus               5000 non-null   float64
 10  Hallucination       5000 non-null   float64
 11  Housing             5000 non-null   object 
 12  Intrusive_Thoughts  4170 non-null   float64
 13  Participant         5000 non-null   int64  
 14  Passive             5000 non-null   float64
 15  Pregnant            2762 non-null   float64
 16  Psycho

In [31]:
data.describe()

,Diagnosis,Anhedonia,Apathy,Appetite,Concentration,Content,Delusion,Dep_Mood,Focus,Hallucination,...,Pregnant,Psychomotor,Rumination,Sleep,Stress,Suspicious,Tension,Tired,Unusual_Thought,Withdrawal
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,2762.000000,5000.000000,5000.000000,5000.000000,5000.000000,2127.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.505800,6.502860,2.478849,27.070029,6.519924,0.279407,2.637389,5.727062,6.519924,65.073832,...,0.101376,4.680583,5.685816,7.011266,4.916418,2.754909,4.920667,inf,2.480266,3.958121
std,0.500016,1.488151,1.730810,14.202618,1.474846,0.834494,1.440347,3.284501,1.474846,223.943469,...,0.301880,1.482692,2.161891,1.410841,2.220262,1.496126,1.962342,NaN,1.410475,1.469444
min,0.000000,1.098854,-3.211011,0.141074,1.299964,0.000187,-2.127037,0.000000,1.299964,0.027350,...,0.000000,-0.024974,-0.409032,2.144726,-3.257788,-2.346238,-2.183456,0.366650,-1.981307,-0.825919
25%,0.000000,5.495361,1.265128,16.724108,5.528181,0.018655,1.629919,4.678095,5.528181,4.113962,...,0.000000,3.697870,4.042552,6.058402,3.443683,1.703462,3.565482,4.491580,1.486439,2.969534
50%,1.000000,6.485527,2.427409,25.165292,6.498042,0.064259,2.558146,6.752196,6.498042,12.764604,...,0.000000,4.720156,5.521805,6.980519,5.096416,2.735108,5.247353,5.513508,2.388994,3.962131
75%,1.000000,7.489218,3.642059,35.447666,7.519759,0.215773,3.588012,8.045706,7.519759,41.814204,...,0.000000,5.682627,7.276673,7.977138,6.531673,3.725759,6.385145,6.569176,3.426667,4.972302
max,1.000000,11.603140,8.803433,113.438734,11.649649,21.001327,8.978785,12.003550,11.649649,6287.163151,...,1.000000,10.171540,12.009666,11.920312,11.970952,8.212275,9.622076,inf,8.066822,9.022207


In [32]:
features = [
 'Anhedonia',
 'Apathy',
 'Appetite',
 'Concentration',
 'Content',
 'Delay',
 'Delusion',
 'Dep_Mood',
 'Focus',
 'Hallucination',
 'Housing',
 'Intrusive_Thoughts',
 'Participant',
 'Passive',
 'Pregnant',
 'Psychomotor',
 'Race',
 'Rumination',
 'Sex',
 'Sleep',
 'Stress',
 'Suspicious',
 'Tension',
 'Tired',
 'Unusual_Thought',
 'Withdrawal']

In [41]:
#split the training and testing datasets
y = data['Diagnosis']
X = data[features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
#X_test_save = X_test


train = pd.concat([X_train,y_train], axis= 1)
test = pd.concat([X_test,y_test], axis= 1)
train = data_remove(train)
test = data_remove(test)


/var/folders/xr/9g352cv10lv4vkg0vccmw9lc0000gn/T/ipykernel_43987/2552075687.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data.fillna(data.mean(), inplace=True)


In [42]:
train.describe()

,Anhedonia,Apathy,Appetite,Concentration,Content,Delusion,Dep_Mood,Focus,Hallucination,Intrusive_Thoughts,...,Psychomotor,Rumination,Sleep,Stress,Suspicious,Tension,Tired,Unusual_Thought,Withdrawal,Diagnosis
count,3453.000000,3453.000000,3453.000000,3453.000000,3453.000000,3453.000000,3453.000000,3453.000000,3453.000000,3453.000000,...,3453.000000,3453.000000,3453.000000,3453.000000,3453.000000,3453.000000,3453.000000,3453.000000,3453.000000,3453.000000
mean,6.507970,2.488079,27.030416,6.509845,0.271884,2.634363,5.694231,6.509845,67.487821,5.723364,...,4.683599,5.695334,7.010393,4.882908,2.740125,4.905160,5.508656,2.480740,3.954453,0.506226
std,1.484693,1.730606,14.081542,1.486728,0.844086,1.447198,3.299082,1.486728,238.685711,2.152243,...,1.476545,2.151656,1.412533,2.229810,0.980364,1.957935,1.495070,1.407826,1.464050,0.500034
min,1.098854,-3.211011,0.141074,1.578187,0.000187,-1.523586,0.000000,1.578187,0.027350,-1.352806,...,0.123560,-0.409032,2.144726,-2.203737,-2.346238,-0.726664,0.687499,-1.981307,-0.825919,0.000000
25%,5.496535,1.279946,16.757048,5.506803,0.018019,1.616857,4.543235,5.506803,4.116464,4.357373,...,3.694768,4.065094,6.058726,3.395626,2.738788,3.545356,4.465110,1.492563,2.978531,0.000000
50%,6.483684,2.438145,25.199759,6.468057,0.063716,2.549434,6.719571,6.468057,12.701013,5.720560,...,4.713601,5.536042,6.982103,5.061072,2.738788,5.208430,5.461856,2.390968,3.942145,1.000000
75%,7.496072,3.633896,35.474871,7.520648,0.211474,3.590477,8.043615,7.520648,43.091082,6.989906,...,5.682510,7.276124,7.972515,6.510192,2.738788,6.372313,6.522500,3.412685,4.966916,1.000000
max,11.603140,8.803433,98.888708,11.649649,21.001327,7.545008,12.003550,11.649649,6287.163151,13.209009,...,10.171540,11.851622,11.920312,11.219529,8.212275,9.562201,11.454125,8.066822,9.022207,1.000000


In [36]:

categories = ['Sex','Race','Housing','Delay']

def onehot(data, categories = categories):
    ordinalencoder = OneHotEncoder()
    onehot = ordinalencoder.fit_transform(data[categories])
    columns = []
    for i, values in enumerate(ordinalencoder.categories_):
        for j in values:
            columns. append(categories[i]+'-'+j)
    return pd.DataFrame(onehot.toarray(), columns = columns)




    


In [44]:
#X_train = X_train.join(onehot(X_train))
#X_test = X_test.join(onehot(X_test))
X_train = pd.get_dummies(train[features], columns = categories)
X_test = pd.get_dummies(test[features], columns = categories)
y_train = train[['Diagnosis']]
y_test = test[['Diagnosis']]
y_train.head()

,Diagnosis
2858,0
1559,1
1441,1
2179,0
1390,1


In [45]:
#training 

model = LogisticRegression(penalty='elasticnet', max_iter= 1000, solver= 'saga', l1_ratio=1)

model.fit(X_train, y_train)

test_predict = model.predict(X_test)

/Users/dahaixing/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/dahaixing/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [46]:
print('test accuracy:', skm.accuracy_score(y_test, test_predict))

test accuracy: 0.8878378378378379


In [47]:
from sklearn.metrics import classification_report
print(classification_report(y_test, test_predict, labels=[1, 0]))

              precision    recall  f1-score   support

           1       0.89      0.89      0.89       741
           0       0.89      0.89      0.89       739

    accuracy                           0.89      1480
   macro avg       0.89      0.89      0.89      1480
weighted avg       0.89      0.89      0.89      1480



In [48]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, test_predict, labels=[1, 0])
print(confusion_matrix)

[[656  85]
 [ 81 658]]


In [17]:
import fairlearn
from fairlearn.metrics import MetricFrame

In [53]:
def race_fp(truelabels, predictions):
  sesitive = test.Race
  fmetrics = MetricFrame(metrics= fairlearn.metrics.false_positive_rate, 
                         y_true=truelabels, 
                         y_pred=predictions,
                         sensitive_features=sesitive)
  results = pd.DataFrame([fmetrics.by_group, fmetrics.by_group/fmetrics.by_group.White], 
                         index= ['FPR', 'FPR Parity'])
  return results

In [54]:
race_fp(y_test, test_predict)

Race,Asian,Black,Hispanic,White
FPR,0.065693,0.181818,0.108527,0.08
FPR Parity,0.821168,2.272727,1.356589,1.00


In [55]:
def raceNsex_fp(truelabels, predictions):
  sesitive = pd.DataFrame(np.stack([test.Race, test.Sex], axis = 1),
                          columns = ['Race','Sex']) 
  fmetrics = MetricFrame(metrics= fairlearn.metrics.false_positive_rate, 
                         y_true=truelabels, 
                         y_pred=predictions,
                         sensitive_features=sesitive)
  results = pd.DataFrame([fmetrics.by_group, fmetrics.by_group/fmetrics.by_group.White.Male], 
                         index= ['FPR', 'FPR Parity'])
  return results

In [56]:
raceNsex_fp(y_test, test_predict)

Race           Asian               Black            Hispanic            \
Sex           Female      Male    Female      Male    Female      Male   
FPR         0.050633  0.086207  0.194969  0.128205  0.090909  0.134615   
FPR Parity  0.416748  0.709549  1.604741  1.055227  0.748252  1.107988   

Race           White            
Sex           Female      Male  
FPR         0.053571  0.121495  
FPR Parity  0.440934  1.000000

In [57]:
def race_fn(truelabels, predictions):
  sesitive = test.Race
  fmetrics = MetricFrame(metrics= fairlearn.metrics.false_negative_rate, 
                         y_true=truelabels, 
                         y_pred=predictions,
                         sensitive_features=sesitive)
  results = pd.DataFrame([fmetrics.by_group, fmetrics.by_group/fmetrics.by_group.White], 
                         index= ['FNR', 'FNR Parity'])
  return results

In [58]:
race_fn(y_test, test_predict)

Race,Asian,Black,Hispanic,White
FNR,0.112500,0.091195,0.138298,0.136546
FNR Parity,0.823897,0.667869,1.012829,1.000000


In [59]:
def raceNsex_fn(truelabels, predictions):
  sesitive = pd.DataFrame(np.stack([test.Race, test.Sex], axis = 1),
                          columns = ['Race','Sex']) 
  fmetrics = MetricFrame(metrics= fairlearn.metrics.false_negative_rate, 
                         y_true=truelabels, 
                         y_pred=predictions,
                         sensitive_features=sesitive)
  results = pd.DataFrame([fmetrics.by_group, fmetrics.by_group/fmetrics.by_group.White.Male], 
                         index= ['FNR', 'FNR Parity'])
  return results

In [61]:
raceNsex_fn(y_test, test_predict)

Race           Asian               Black            Hispanic            \
Sex           Female      Male    Female      Male    Female      Male   
FNR         0.139535  0.081081  0.186916  0.042654  0.204082  0.066667   
FNR Parity  1.674419  0.972973  2.242991  0.511848  2.448980  0.800000   

Race           White            
Sex           Female      Male  
FNR         0.177305  0.083333  
FNR Parity  2.127660  1.000000